In [12]:
from bs4 import BeautifulSoup
import requests, json, os
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import pprint
import gensim

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.52",
}

params = {
    "q": "",
    "hl": "en",
    "start": 0
}
from tkinter import *
window=Tk()


link = []

In [ ]:
#SPARK RELATED CODE
import findspark

findspark.init()

from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession \
    .builder \
    .appName('Sparkler') \
    .getOrCreate()

spark.conf.set('spark.sql.caseSensitive', False)

In [13]:
def gensimImplementation(documents):
    q = params['q']
    for document in documents:
        print(gensim.utils.simple_preprocess(document))

In [14]:
#Similar Articles
def getSimilarArticles(df, vectorizer):    
    q = [params['q']]
    q_vector = vectorizer.transform(q).toarray().reshape(df.shape[0],)
    print(q_vector)
    sim = {}
    
    for i in range(df.shape[1]):
        sim[i] = np.dot(df.loc[:, i].values, q_vector) / np.linalg.norm(df.loc[:,i]) * np.linalg.norm(q_vector)
        
    sim_sorted = sorted(sim.items(), key = lambda x:x[1], reverse = True)
    print(sim_sorted)
    for k,v in sim_sorted[:5]:
        if v != 0.0:
            print("Similarities: {}".format(v))
            print('Link to the article:', link[k])

In [15]:
def vectorizerMethod(documents_clean):
    # Instantiate a TfIdfVectorizer Object and transform the data to vectors
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(documents_clean)
    X = X.T.toarray()

    df = pd.DataFrame(X, index=vectorizer.get_feature_names_out())
    getSimilarArticles(df, vectorizer)
    

In [16]:
def cleanData(documents):
    documents_clean = []
    for d in documents:
        # Remove Unicode
        document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
        # Remove Mentions
        document_test = re.sub(r'@\w+', '', document_test)
        # Lowercase the document
        document_test = document_test.lower()
        # Remove punctuations
        document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
        # Lowercase the numbers
        document_test = re.sub(r'[0-9]', '', document_test)
        # Remove the doubled space
        document_test = re.sub(r'\s{2,}', ' ', document_test)
        documents_clean.append(document_test)
    vectorizerMethod(documents_clean)
#     gensimImplementation(documents=documents_clean)


In [17]:
def collectDocumentFromLinks(link):
    #Retrieve paragrahs from each link, combine each paragrah as a string and save it to docs
    documents = []
    print('Fetching data from each link...')
    for i in link:
        r = requests.get(i, headers=headers)
        soup = BeautifulSoup(r.content,'html.parser')

        sen = []
        # for springer abstracts
        if soup.find('div', {'id':'Abs1-content'}):
            for i in soup.find('div', {'id':'Abs1-content'}).find_all('p'):
                sen.append(i.text)

        documents.append(' '.join(sen))
    print('Fetched data from each link...')
    cleanData(documents)

In [18]:
def collectSpringerLinks(soup):
    #Retrieve all popular new links
    print(len(soup))
    i=0
    for i in range(0, len(soup)):
        data = soup[i].find_all("div", {"class": "gs_ri"})
        for j in range(len(data)):    

            temp = data[j].find('a')

            if 'link.springer.com/article' in temp['href'] and 'books.google.com' not in temp['href']:
                link.append(temp['href'])
    #             print(temp['href'], j)

    print(len(link))
    if len(link)>0:
        collectDocumentFromLinks(link)
    else: 
        print('No links found. Try with other keywords.')

In [19]:
def collectGoogleScholarPages(query):
    params['q'] = query
    soup = []
    while True:
        url = 'https://scholar.google.com/scholar'
        req = requests.get(url, headers=headers, params=params)
        print(req.url)
        tempData = BeautifulSoup(req.content,'html.parser')
#         print(tempData)
        soup.append(tempData)
        if tempData.find('span', {'class': 'gs_ico gs_ico_nav_next'}) and params['start']<=100:
            params['start']+=10
        else:
            break
    # call collectSpringerLinks()
    collectSpringerLinks(soup)
    

In [20]:
# lbl=Label(window, text="Enter keywords to search documents", fg='black', font=("Helvetica", 12))
# lbl.place(relx=.5, rely=.25,anchor= CENTER)


# txtfld=Entry(window, text="Enter keywords (',' separated)", bd=2)
# txtfld.place(relx=.5, rely=.35,anchor= CENTER)

# btn=Button(window, text="Search Documents", fg='black', command=lambda: collectGoogleScholarPages(txtfld.get()))
# btn.place(relx=.5, rely=.45,anchor= CENTER)

# window.title('Sparkler for Documents')
# window.geometry("500x300+250+250")
# window.mainloop()


keywords = input("Enter keywords on which you want to search documents:")
collectGoogleScholarPages(keywords)

Enter keywords on which you want to search documents:brain tumor springer covid-19
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=0
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=10
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=20
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=30
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=40
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=50
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=60
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=70
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=80
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=90
https://scholar.google.com/scholar?q=brain+tumor+springer+covid-19&hl=en&start=100
https://scholar

C:\Users\babap\AppData\Local\Temp\ipykernel_24100\3896186481.py:9: RuntimeWarning: invalid value encountered in double_scalars
  sim[i] = np.dot(df.loc[:, i].values, q_vector) / np.linalg.norm(df.loc[:,i]) * np.linalg.norm(q_vector)
